In [13]:
import pandas as pd
from sklearn.metrics import accuracy_score

In [14]:
df_train = pd.read_csv("data/train.csv")
df_artists = pd.read_csv("data/artists.csv")

In [15]:
seeds = [85, 25, 121, 61, 73, 37, 97, 49, 109, 13]

for seed in seeds:
    predictions = pd.read_csv(f"data/predictions/final/final_seed{seed}.csv")
    df_merged = predictions.merge(df_train, on=["song_id", "msno"], how="left")
    y_pred = df_merged['probability'] >= 0.5
    y_true = df_merged['target']
    print(accuracy_score(y_true, y_pred))

0.7652216424176081
0.7663708600356385
0.7657353186032259
0.7658734797841852
0.7656254981773352
0.7661965643919668
0.7659188249922949
0.7664048689417208
0.7662652907230081
0.767075836317969
